In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
!pip install python-dotenv langchain_community openAI langchain_core langchain langchain_openai langchain_chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.6 MB/s eta 0:00:00


In [3]:
# 환경변수 설정

In [4]:
# 라이브러리 불러오기
from dotenv import load_dotenv
import os
from langchain_openai import OpenAI

In [5]:
# .env 파일에서 환경 변수 로드
load_dotenv("/content/drive/MyDrive/Colab Notebooks/.env")
# 환경 변수에서 API 키 가져오기
api_key = os.getenv("OPENAI_API_KEY")
# 오픈AI 대규모 언어 모델 초기화
llm = OpenAI()

In [6]:
# 예제 1-24 <이전 대화를 포함한 메시지 전달>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-4o-mini")

# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        ("placeholder", "{messages}"),  # 대화 이력 추가
    ]
)

# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | chat

# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),  # 사용자의 첫 질문
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."),  # 챗봇의 답변
            ("human", "방금 뭐라고 했나요?"),  # 사용자의 재확인 질문
        ],
    }
)
print(ai_msg.content)  # 챗봇의 응답 출력


저축을 늘리기 위해 저축 목표를 설정하고 매달 자동 이체로 일정 금액을 저축하는 것이 좋다고 말씀드렸습니다. 이렇게 하면 저축을 습관화하고 더 쉽게 목표에 도달할 수 있습니다. 추가적인 방법이나 조언이 필요하시면 말씀해 주세요!


In [9]:
# 예제 1-25 <`ChatMessageHistory`를 사용한 메시지 관리>

from langchain_community.chat_message_histories import ChatMessageHistory

# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()

# 사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})
print(ai_response.content)  # 챗봇은 이전 메시지를 기억하여 답변합니다.

저축을 늘리기 위해서는 먼저 저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하는 방법을 추천합니다. 이를 통해 규칙적으로 저축할 수 있고, 저축 습관을 기를 수 있습니다. 추가로 예산을 세워 불필요한 지출을 줄이는 것도 도움이 됩니다.


In [10]:
# 예제 1-26 <` RunnableWithMessageHistory`를 사용한 메시지 관리>

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory

# 시스템 메시지와 대화 이력을 사용하는 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 모든 질문에 최선을 다해 답변하십시오."),
        ("placeholder", "{chat_history}"),  # 이전 대화 이력
        ("human", "{input}"),  # 사용자의 새로운 질문
    ]
)

# 대화 이력을 관리할 체인 설정
chat_history = ChatMessageHistory()
chain = prompt | chat

# RunnableWithMessageHistory 클래스를 사용해 체인을 감쌉니다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따라 대화 이력을 불러오는 함수
    input_messages_key="input",  # 입력 메시지의 키 설정
    history_messages_key="chat_history",  # 대화 이력의 키 설정
)

# RunnableWithMessageHistory 클래스를 사용해 체인을 감쌉니다
chain_with_message_history = RunnableWithMessageHistory(
    chain,
    lambda session_id: chat_history,  # 세션 ID에 따라 대화 이력을 불러오는 함수
    input_messages_key="input",  # 입력 메시지의 키 설정
    history_messages_key="chat_history",  # 대화 이력의 키 설정
)

# 질문 메시지 체인 실행
chain_with_message_history.invoke(
    {"input": "저축을 늘리기 위해 무엇을 할 수 있나요?"},
    {"configurable": {"session_id": "unused"}},
).content


'저축을 늘리기 위해 다음과 같은 방법들을 고려해 볼 수 있습니다:\n\n1. **예산 세우기**: 수입과 지출을 분석하여 예산을 세우고, 필요 없는 지출을 줄이는 것이 중요합니다. 매달 얼마를 저축할 수 있는지 계획해 보세요.\n\n2. **자동 이체 설정**: 월급이 들어오면 자동으로 저축 계좌로 일정 금액을 이체되도록 설정하여 저축을 습관화하세요.\n\n3. **비상금 마련**: 예기치 않은 지출을 대비해 비상금을 마련하면 일반적인 저축에 더 집중할 수 있습니다.\n\n4. **소비 습관 점검**: 불필요한 소비를 줄이기 위해 소비 습관을 점검하고, 충동구매를 피하는 방법을 모색하세요.\n\n5. **목표 설정**: 단기, 중기, 장기 저축 목표를 설정하고, 그 목표에 맞춰 저축 계획을 세워 보세요. 목표가 있을 때 더 동기부여가 됩니다.\n\n6. **고금리 저축 계좌 이용**: 가능한 한 높은 이율을 제공하는 저축 계좌를 찾아서 이자를 통해 더 많은 돈을 저축하세요.\n\n7. **정기적으로 저축금액 검토**: 생활비나 수입 변화에 따라 저축금액을 정기적으로 검토하고 조정하세요.\n\n8. **부가 수입 창출**: 프리랜스 작업, 아르바이트, 또는 취미를 활용한 수익 창출 등을 통해 추가적인 수입을 만들어 저축할 수 있습니다.\n\n9. **세금 공제 및 혜택 활용**: 세금 혜택이 있는 저축 상품이나 투자 상품을 활용하여 절세와 저축을 동시에 할 수 있습니다.\n\n이와 같은 방법들을 통해 저축을 늘릴 수 있습니다. 각자의 상황에 맞는 전략을 찾아 실천해 보세요!'

In [11]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_message_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content


'당신은 "저축을 늘리기 위해 무엇을 할 수 있나요?"라고 질문하셨습니다. 저축을 늘리기 위한 여러 가지 방법에 대해 설명드렸습니다. 추가로 궁금한 점이 있으시면 언제든지 말씀해 주세요!'

In [14]:
# 예제 1-27 <메시지 트리밍 예제>

# 라이브러리 불러오기
from langchain_core.messages import trim_messages
from langchain_core.runnables import RunnablePassthrough
from operator import itemgetter

# 메시지 트리밍 유틸리티 설정
trimmer = trim_messages(strategy="last", max_tokens=2, token_counter=len)

# 트리밍된 대화 이력과 함께 체인 실행
chain_with_trimming = (
    RunnablePassthrough.assign(chat_history=itemgetter("chat_history") | trimmer)
    | prompt
    | chat
)

# 트리밍된 대화 이력을 사용하는 체인 설정
chain_with_trimmed_history = RunnableWithMessageHistory(
    chain_with_trimming,
    lambda session_id: chat_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

# 새로운 대화 내용 추가 후 체인 실행
chain_with_trimmed_history.invoke(
    {"input": "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
)



AIMessage(content='5년 내에 집을 구매하기 위해서는 체계적인 재정 계획이 중요합니다. 다음은 그 과정을 단계별로 정리한 것입니다:\n\n1. **구매 목표 설정**:\n   - 원하는 주택의 가격대와 지역을 정하세요. 이를 통해 필요한 자금의 규모를 파악할 수 있습니다.\n\n2. **예산 수립**:\n   - 현재의 재정 상태를 검토하세요. 월 수입과 지출을 정리하고, 얼마나 저축할 수 있는지를 계산합니다.\n\n3. **저축 계획**:\n   - 구매 목표 금액에 따라 매달 얼마를 저축해야 할지 계산합니다. 예를 들어, 5년 후 3천만 원의 집을 구매하고자 한다면, 적어도 1천만 원의 계약금을 모아야 할 수 있습니다. 5년 동안 매달 약 16만 원을 저축해야 합니다.\n\n4. **비상금 마련**:\n   - 집을 구매하기 전, 일정 금액의 비상금을 마련해 두는 것이 좋습니다. 일반적으로 3~6개월치의 생활비를 비상금으로 준비하는 것이 권장됩니다.\n\n5. **투자 고려**:\n   - 저축만으로는 목표 금액을 모으기 어려울 수 있습니다. 적절한 투자 방법을 고려해 보세요. 예를 들어, 저축은행의 정기예금, 상장지수펀드(ETF), 주식 등을 통해 자산을 늘릴 수 있습니다.\n\n6. **신용 점수 관리**:\n   - 주택 구매 시 대출이 필요할 경우, 신용 점수가 중요합니다. 신용 점수를 높이기 위해 카드 대금을 제때 갚고, 불필요한 부채를 줄이는 것이 좋습니다.\n\n7. **주택담보대출 사전 승인**:\n   - 대출 가능성을 미리 확인해 보세요. 대출 한도를 알고 있으면 예산을 더 잘 세울 수 있습니다.\n\n8. **전문가 상담**:\n   - 필요시 금융 상담사나 주택 구매 전문가와 상담하여 구체적인 조언을 받을 수 있습니다.\n\n이러한 계획을 세우고 실행에 옮기면, 5년 내에 집을 사는 목표에 한 걸음 더 다가갈 수 있을 것입니다. 추가 질문이 있으시면 언제든지 말씀해 주세요!', additional_kwargs={'refu

In [15]:
# 새로운 입력 메시지를 추가하고 체인을 실행
chain_with_trimmed_history.invoke(
    {"input": "내가 방금 뭐라고 했나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "finance_session_1"}}  # 세션 ID 설정
).content


'당신은 "저는 5년 내에 집을 사기 위해 어떤 재정 계획을 세워야 하나요?"라고 말씀하셨습니다. 이 질문에 대해 제가 재정 계획을 세우기 위한 단계별 조언을 드렸습니다. 추가 질문이나 다른 도움이 필요하시면 말씀해 주세요!'

In [16]:
# 예제 1-28 <이전 대화 요약 내용 기반으로 답변하기>

def summarize_messages(chain_input):
    stored_messages = chat_history.messages
    if len(stored_messages) == 0:
        return False
    # 대화를 요약하기 위한 프롬프트 템플릿 설정
    summarization_prompt = ChatPromptTemplate.from_messages(
        [
            ("placeholder", "{chat_history}"),  # 이전 대화 이력
            (
                "user",
                "이전 대화를 요약해 주세요. 가능한 한 많은 세부 정보를 포함하십시오.",  # 요약 요청 메시지
            ),
        ]
    )

    # 요약 체인 생성 및 실행
    summarization_chain = summarization_prompt | chat
    summary_message = summarization_chain.invoke({"chat_history": stored_messages})
    chat_history.clear()  # 요약 후 이전 대화 삭제
    chat_history.add_message(summary_message)  # 요약된 메시지를 대화 이력에 추가
    return True

In [18]:
# 대화 요약을 처리하는 체인 설정
chain_with_summarization = (
    RunnablePassthrough.assign(messages_summarized=summarize_messages)
    | chain_with_message_history
)

# 요약된 대화를 기반으로 새로운 질문에 응답
print(chain_with_summarization.invoke(
    {"input": "저에게 어떤 재정적 조언을 해주셨나요?"},  # 사용자의 질문
    {"configurable": {"session_id": "unused"}}  # 세션 ID 설정
).content)


아래는 제가 이전에 드린 재정적 조언의 요약입니다:

1. **저축 늘리기 방법**:
   - **예산 세우기**: 수입과 지출을 정리하여 어디서 절약할 수 있는지 파악하세요.
   - **소비 습관 점검**: 필요 없는 지출을 줄이고, 필수 지출에 집중하세요.
   - **자동 이체 설정**: 매달 정해진 금액을 저축 계좌로 자동 이체하여 저축을 습관화하세요.
   - **비상금 마련**: 예상치 못한 지출을 대비할 수 있도록 비상금을 준비하세요.
   - **목표 설정**: 구체적인 저축 목표를 설정하여 동기 부여를 높이세요.
   - **고금리 저축 계좌 이용**: 높은 이자를 제공하는 저축 계좌를 찾아보세요.
   - **부가 수입 창출**: 추가적인 수입을 얻을 수 있는 방법을 모색하세요.
   - **세금 공제 및 혜택 활용**: 세금 관련 혜택을 최대한 활용하세요.

2. **5년 내 집 구매를 위한 재정 계획**:
   - **구매 목표 설정**: 원하는 주택 가격과 지역을 정하고, 필요한 자금을 파악하세요.
   - **예산 수립**: 현재 재정 상태를 분석하고, 저축할 수 있는 금액을 계산하세요.
   - **저축 계획**: 목표 금액에 따라 매달 저축해야 할 금액을 정하세요.
   - **비상금 마련**: 생활비의 3~6개월치를 비상금으로 준비하세요.
   - **투자 고려**: 저축 외에도 장기적으로 투자할 수 있는 방법을 검토하세요.
   - **신용 점수 관리**: 대출을 계획하고 있다면 신용 점수를 관리하세요.
   - **주택담보대출 사전 승인**: 대출 가능한 한도를 미리 확인하세요.
   - **전문가 상담**: 필요한 경우 금융 전문가와 상담하여 조언을 받으세요.

이런 조언들이 재정적 목표 달성에 도움이 되기를 바랍니다. 추가적인 질문이 있으시면 언제든지 말씀해 주세요!
